In [2]:
pip install numpy pandas matplotlib scikit-learn tensorflow yfinance joblib

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import yfinance as yf

In [4]:
# Download historical stock data
ticker = 'AAPL'  # Apple stock
start_date = '2010-01-01'
end_date = '2023-12-31'
df = yf.download(ticker, start=start_date, end=end_date)


/tmp/ipython-input-4-4209640893.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


In [5]:
# Preprocess data
data = df['Close'].values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [6]:
# Create training dataset
training_len = int(len(scaled_data) * 0.8)
train_data = scaled_data[0:training_len, :]

In [7]:
# Create sequences
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i, 0])
        y.append(data[i, 0])
    return np.array(X), np.array(y)

SEQ_LENGTH = 60
X_train, y_train = create_sequences(train_data, SEQ_LENGTH)

# Reshape data for LSTM [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [8]:
# Build LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(25))
model.add(Dense(1))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train model
early_stop = EarlyStopping(monitor='loss', patience=5, verbose=1)
history = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=100,
    callbacks=[early_stop]
)

# Save model
model.save('stock_prediction_model.h5')
print("Model saved as 'stock_prediction_model.h5'")

# Save scaler for future use
import joblib
joblib.dump(scaler, 'scaler.pkl')
print("Scaler saved as 'scaler.pkl'")

# Plot training loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'])
plt.title('Model Training Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.savefig('training_loss.png')
plt.close()

print("Model training complete!")

Epoch 1/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.0045
Epoch 2/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 4.8946e-04
Epoch 3/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 4.1133e-04
Epoch 4/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 5.1697e-04
Epoch 5/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 4.6165e-04
Epoch 6/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 3.2473e-04
Epoch 7/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 3.1254e-04
Epoch 8/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 2.9960e-04
Epoch 9/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 2.6272e-04
Epoch 10/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 3.5161e-04
Epoch 11/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 3.2336e-04
Epoch 12/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 3.3395e-04
Epoch 13/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.5115e-04
Epoch 14/100
87/87 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 2.8703e-04
Epoch 15/100
87/8

Model saved as 'stock_prediction_model.h5'
Scaler saved as 'scaler.pkl'
Model training complete!
